In [86]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
data_df = pd.read_csv('IMDB Dataset.csv')

In [34]:
MAX_LEN = 512
PAD_INDEX = 0
BATCH_SIZE = 16

In [5]:
def tokenize(text):
    token = torch.tensor(tokenizer(text, truncation=True)['input_ids'])
    # колхозные паддинги
    padded_token = torch.cat((token, torch.zeros(MAX_LEN - token.shape[0])), 0)
    return padded_token

In [6]:
data = list(zip(
    list(map(tokenize, data_df['review'])),
    list(map(lambda sent: 0 if 'negative' else 1, data_df['sentiment'])),
))

In [41]:
train = data[:-5000]
print(len(train))
test = data[-5000:]
print(len(test))

45000
5000


In [42]:
# проверяем, что паддинги сработали корректно
all(map(lambda t: t[0].shape[0] == MAX_LEN, train))

True

In [43]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE)

In [44]:
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

In [45]:
def train(train_loader, test_loader, criterion, optimizer):
    pass

In [94]:
model = AutoModel.from_pretrained("bert-base-uncased").to('cuda:0')

In [95]:
for p in model.parameters():
    p.requires_grad = False

In [98]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
criterion = torch.nn.CrossEntropyLoss()
classifier = torch.nn.Linear(768, 2).to('cuda:0')

In [100]:
losses = []
train_losses_epoch = []
for x, y in tqdm(train_loader):
    pred = model(x.long().to('cuda:0'))
    pred = classifier(pred.pooler_output)
    train_loss = criterion(pred, y.to('cuda:0'))
    train_loss.backward()
    train_losses_epoch.append(train_loss.item())
    optimizer.step()
    optimizer.zero_grad()

  2%|▏         | 50/2813 [01:01<56:43,  1.23s/it] 


KeyboardInterrupt: 

In [84]:
for p in model.parameters():
    p.requires_grad = False

In [77]:
o = model(x.long())

In [78]:
classifier = torch.nn.Linear(768, 2)

In [79]:
o.pooler_output.shape

torch.Size([16, 768])

In [81]:
p = l(o.pooler_output)

In [83]:
print(p.shape, y.shape)

torch.Size([16, 2]) torch.Size([16])
